In [136]:
# !pip install hydrafloods
# !pip install eepackages
# !pip install folium

import hydrafloods as hf
import ee
import eepackages 
import folium
from folium import plugins
from eepackages import assets
from eepackages import utils

In [137]:
# initialize GEE
# ee.Authenticate()
ee.Initialize()

In [138]:
start_time = "2019-10-05"
end_time   =  "2019-10-06"
region     = ee.Geometry.Rectangle([104, 11.5, 106, 12.5 ])

s1 = hf.datasets.Sentinel1(region, start_time, end_time)

water_imgs = s1.apply_func(
                            hf.thresholding.edge_otsu,
                            initial_threshold=-14,
                            thresh_no_data=-20,
                            edge_buffer=300
                          )

water_map = ee.Image(water_imgs.collection.mode())

# hf.geeutils.export_image(
#     water_map,
#     region,
#     "users/mafmonjaraz/water_map_example",
#     scale=30,
# )

In [139]:
def getCollection(aoi, startyear,endyear):
    
    def clip(image):
        return image.clip(aoi)
    
    period = ee.Filter.date(
            str(startyear)   + '-09-01',
            str(endyear + 1) + '-09-01')

    collection = assets.getMostlyCleanImages(assets.getImages(aoi,
            {
                'resample': True,
                'filter': period,
                'filterMasked': True,
                'filterMaskedFraction': 0.9,
                'missions': ['L4', 'L5', 'L8', 'L9', 'S2']
            }
            ), 
            aoi, 
            {
                'percentile': 98,
                'qualityBand': 'blue',
            }).map(clip)
        
    return collection

def getIndexes(collection):
    def calculateIndexes(image):
        ndwi = image.normalizedDifference(['green', 'nir']).select('nd').rename('ndwi')
        ndvi = image.normalizedDifference(['nir', 'red']).select('nd').rename('ndvi')
        return image.addBands(ndwi, ndvi)
    return collection.map(calculateIndexes)


def exportYearly(yr, collection, region):    
    ts = ee.Date.fromYMD(yr, 9, 1)
    te = ts.advance(1, 'year')
    yearimg = collection.filterDate(ts, te).mean().set({
        'date': ts,
        'year': yr
    })
    task = ee.batch.Export.image.toAsset(
        image = yearimg,
        description= 'eflows-demo-tenryu-' + str(yr),
        # assetId= 'users/deltareseflows/tenryu-IAHR/' + 'tenryu-' + str(yr) + '-ndwi',
        assetId= 'users/mafmonjaraz/' + 'tenryu-' + str(yr) + '-ndwi',
        region= region,
        scale=30,
        crs= 'EPSG:32653',
        maxPixels= 1e13)
    task.start()
    return
                         

In [140]:
startyear= 2022
endyear  = 2023
monthList = ee.List.sequence(1,12,1)
yearList = ee.List.sequence(startyear, endyear, 1)
tenryu   = ee.FeatureCollection("users/deltareseflows/tenryu-basin-bounds")
geometry = ee.Geometry.Polygon(
        [[[137.79394768834086, 34.86437147904835],
          [137.79669427037211, 34.853947959534395],
          [137.8066506302354, 34.85056707495847],
          [137.82038354039165, 34.85056707495847],
          [137.8238167679307, 34.834505976178946],
          [137.81695031285258, 34.80603914064101],
          [137.8172936356065, 34.7894054479401],
          [137.81145714879008, 34.77220165226879],
          [137.80253075718852, 34.74371329912516],
          [137.7905144608018, 34.70844182876761],
          [137.78673791050883, 34.69150617819119],
          [137.77815484166118, 34.659883691297125],
          [137.7798714554307, 34.641525818990495],
          [137.82244347691508, 34.642090737217515],
          [137.8238167679307, 34.65734207454119],
          [137.80596398472758, 34.660448484435065],
          [137.7963509476182, 34.664401928616506],
          [137.79875420689555, 34.68219009300521],
          [137.80459069371196, 34.701385728940906],
          [137.80905388951274, 34.71549732675007],
          [137.81386040806743, 34.73412094926915],
          [137.82107018589946, 34.74371329912516],
          [137.82519005894633, 34.762612611120225],
          [137.83136986851665, 34.78348145078419],
          [137.82759331822368, 34.79701517323708],
          [137.83239983677836, 34.81505668123394],
          [137.83342980504008, 34.833657849541886],
          [137.83033990025493, 34.855072172059536],
          [137.81351708531352, 34.86183343202275],
          [137.81454705357524, 34.87929744779061],
          [137.79223107457133, 34.880142386722035]]])
lower    = geometry.intersection(tenryu.geometry(),1)


In [141]:
imageCollection   = getCollection(geometry, startyear, endyear)
single_image       = imageCollection.select('green').first()

for year in range(startyear,endyear+1):
    exportYearly(year,imageCollection, lower)
  

In [142]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:    

        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True,
            style_function=lambda x:vis_params
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


In [143]:
lon_lat         =  [137.79394768834086, 34.86437147904835]
my_map = folium.Map(location= [lon_lat[1], lon_lat[0]], zoom_start=12)
basemaps['Esri Satellite'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)
vis    = {"min":0, "max":5 ,"palette":["ff4545","a7ff7a","1f1e6e"]}
# my_map.add_ee_layer(region, {}, 'region')
# my_map.add_ee_layer(water_map,vis, 'my_map')

my_map.add_ee_layer(geometry, {}, 'geometry')
my_map.add_ee_layer(single_image,{}, 'single_image')


my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

display(my_map)
